In [88]:
using Distributions
using RCall
@rimport stats as stats

In [34]:
function is_symmetric_Hypergeometric(s, f, n)
    fnchg = FisherNoncentralHypergeometric(s, f, n, 1.0)
    xmin, xmax = extrema(fnchg)
    for i in 0:(xmax-xmin)÷2
        pdf(fnchg, xmin+i) != pdf(fnchg, xmax-i) && return false
    end
    true
end

is_symmetric_Hypergeometric (generic function with 1 method)

In [83]:
mn = [(m, n) for m in 1:10 for n in 1:2m-1 if !is_symmetric_Hypergeometric(m, m, n)]
@show mn;

mn = [(6, 6), (7, 3), (7, 6), (7, 7), (7, 8), (7, 12), (8, 5), (8, 7), (8, 8), (8, 9), (9, 6), (9, 7), (9, 8), (9, 11), (9, 12), (9, 13), (9, 14), (9, 15), (10, 2), (10, 4), (10, 5), (10, 8), (10, 9), (10, 12), (10, 13), (10, 15), (10, 18)]


In [84]:
@show m, n = mn[20]
fnchg = FisherNoncentralHypergeometric(m, m, n, 1.0)
[(k, pdf(fnchg, k)) for k in support(fnchg)]

(m, n) = mn[20] = (10, 4)


5-element Vector{Tuple{Int64, Float64}}:
 (0, 0.043343653250773995)
 (1, 0.2476780185758514)
 (2, 0.4179566563467493)
 (3, 0.2476780185758514)
 (4, 0.04334365325077399)

In [95]:
function pvalue_less_equal_(a, b, c, d)
    fnchg = FisherNoncentralHypergeometric(a+b, c+d, a+c, 1.0)
    sum(pdf(fnchg, x) for x in support(fnchg) if pdf(fnchg, x) ≤ pdf(fnchg, a))
end

x ⪅ y = x < y || x ≈ y

function pvalue_less_approx(a, b, c, d)
    fnchg = FisherNoncentralHypergeometric(a+b, c+d, a+c, 1.0)
    sum(pdf(fnchg, x) for x in support(fnchg) if pdf(fnchg, x) ⪅ pdf(fnchg, a))
end

function pvalue_fisher_test(a, b, c, d)
    rcopy(stats.fisher_test([a b; c d]))[:p_value]
end

pvalue_fisher_test (generic function with 1 method)

In [96]:
@show m, n = mn[20]
a = 4
b, c, d = m-a, n-a, m+a-n
@show a, b, c, d
@show pvalue_less_equal_(a, b, c, d)
@show pvalue_less_approx(a, b, c, d)
@show pvalue_fisher_test(a, b, c, d);

(m, n) = mn[20] = (10, 4)
(a, b, c, d) = (4, 6, 0, 10)
pvalue_less_equal_(a, b, c, d) = 0.04334365325077399
pvalue_less_approx(a, b, c, d) = 0.08668730650154799
pvalue_fisher_test(a, b, c, d) = 0.08668730650154799
